In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import scipy
import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc
from Util1 import DataManipulation
from matplotlib import colors
import matplotlib

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.pipeline      import make_pipeline

from sklearn.metrics import r2_score

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import  GridSearchCV

from sklearn.model_selection import learning_curve



# DATA MANIPULATION:

In [ ]:
Data = pd.read_excel(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3\Geophysical_Logs_Well_1.xlsx', sheet_name='Well_1')
Data

Remove Missing Data

In [ ]:
m = DataManipulation(Data)
m.VisualizeMissingData()
m.MissingDataSummarizer()

In [ ]:
Data_Manipulated = m.DropMissingData()
m.VisualizeMissingData()
m.MissingDataSummarizer()

Remove Outliers

In [ ]:
m.CrossPlot()
m.BoxPlot()

In [ ]:
outlier_indexes = []
for i in np.array(Data_Manipulated.columns):
    if not i in ['Water Saturation (SW)']:
        data_tukey = Data_Manipulated[i]
        outliers = m.tukey_outliers(data_tukey, Data_Manipulated.index)[0]
        idx = m.tukey_outliers(data_tukey, Data_Manipulated.index)[1]
        outlier_indexes.append(idx)
        print(i, outliers)
        plt.hist(data_tukey, bins=50, color='gray', alpha=0.5)
        plt.xlabel(i)
        plt.ylabel('N')
        plt.title("Outlier Detection")
        plt.scatter(outliers, [0 for j in range(len(outliers))], color='red')
        plt.show()

Conbining the results from Tukey's outlier detection and Question 3, we can be sure that the numbers greater than or equal to 999 are outliers

UPDATE: I decided to remove all the outliers that are dedected by TukeyFunction to improve regression prediction

In [ ]:
outlier_indexes = np.hstack(outlier_indexes)

In [ ]:
outlier_indexes =  np.unique(outlier_indexes)

In [ ]:
Data_Manipulated = Data_Manipulated.drop(outlier_indexes)

In [ ]:
Data_Manipulated = Data_Manipulated[Data_Manipulated['Water Saturation (SW)']<999]

In [ ]:
n= 0
for i in Data_Manipulated['Water Saturation (SW)']:
    if i == 1:
        n+=1
n # Number of SW = 1.0

In [ ]:
plt.plot(Data_Manipulated['Water Saturation (SW)'], 'r')

In [ ]:
m = DataManipulation(Data_Manipulated)
m.CrossPlot()
m.BoxPlot()

In [ ]:
m.MissingDataSummarizer()

In [ ]:
m.DataAnalysisResults()

# Question 1: Using the data (given as a separate file Geophysical_Logs_Well_1.xlsx), carry out the following analysis

1. Use atleast two unsupervised learning techniques (Kmeans, DBscan, Hierarchical or Geometric means) to make appropriate number of clusters.
2. Visualize and find the value of Silhouette coefficient.
3. Improve the value of Silhouette coefficient by changing the number of clusters.

In [ ]:
# Data scaling
#INscaler=StandardScaler()
#INDataScaled = INscaler.fit(Data_Manipulated.drop(['Water Saturation (SW)'], axis = 1))
#OUTscaler=StandardScaler()
#OUTDataScaled = OUTscaler.fit(np.array(Data_Manipulated['Water Saturation (SW)']).reshape((-1, 1)))

Saving Scaler to add pipeline

In [ ]:
from joblib import dump, load

# save the model to a file
#dump(INscaler, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\INscaler.joblib')
#dump(OUTscaler, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\OUTscaler.joblib')
# load the saved model from a file
INscaler = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\INscaler.joblib')
OUTscaler = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\OUTscaler.joblib')

INDataScaled = INscaler.transform(Data_Manipulated.drop(['Water Saturation (SW)'], axis = 1))
OUTDataScaled = OUTscaler.transform(np.array(Data_Manipulated['Water Saturation (SW)']).reshape((-1, 1)))

In [ ]:
class WellLog_Based_Predictor:
    def __init__(self, clusterer_Data):
        '''clusterer_Data = {'name' : 'KMeans', 'n_clusters': 5, 'random_state': 1000, 'init': 'k-means++', 'n_init':1000, 'max_iter':500} or {'name':'DBSCAN', 'eps': 0.9, 'min_samples': 24, 'metric':'euclidean'} or {'name' : 'GMM', 'n_clusters': 3}'''
        self.clusterer_Data = clusterer_Data

    def Clusterer(self, Data):
        if self.clusterer_Data['name'] == 'KMeans':
            n_clusters   = self.clusterer_Data['n_clusters']
            random_state = self.clusterer_Data['random_state']
            init         = self.clusterer_Data['init']
            n_init       = self.clusterer_Data['n_init']
            max_iter     = self.clusterer_Data['max_iter']
            # Initialize the Clusterer
            kmeans = KMeans(n_clusters= n_clusters, random_state= random_state, init=init, n_init= n_init, max_iter= max_iter)
            # Fitting with inputs
            kmeans = kmeans.fit(INDataScaled)
            # print location of clusters learned by kmeans object
            centers = kmeans.cluster_centers_
            # Extract Labels
            labels = kmeans.labels_
            # inertia
            inertia = kmeans.inertia_
            return centers, labels, inertia

        elif self.clusterer_Data['name'] == 'DBSCAN':
            eps         = self.clusterer_Data['eps']
            min_samples = self.clusterer_Data['min_samples']
            metric      = self.clusterer_Data['metric']
            # Initialize the Clusterer
            Clustering= DBSCAN(eps= eps, min_samples=min_samples, metric=metric)
            # Fit with input Data and Extract Labels
            labels = Clustering.fit_predict(Data)
            return labels

        elif self.clusterer_Data['name'] == 'GMM':
            n_clusters   = self.clusterer_Data['n_clusters']
            # Initialize and Fit to Thge Data
            gmm = GaussianMixture(n_components=int(n_clusters)).fit(Data)
            # Predict Labels
            labels = gmm.predict(Data)
            return labels

        else:
            print('Entered Clusterer is Not Found')
            print("Selections: DBSCAN KMeans GMM")

## Silhouiette Coefficient Calculator

In [ ]:
def silhouette_coeff(labels, n, epsilon, data, clusterer = 'KMeans'):
    n_clusters=np.unique(labels).shape[0]
    silhouette_vals=silhouette_samples(data,labels,metric='euclidean')
    y_ax_lower, y_ax_upper=0,0

    yticks=[]

    plt.figure(figsize=(20, 15))

    for i, c in enumerate (np.unique(labels)):
        c_silhouette_vals= silhouette_vals[labels==c]
        c_silhouette_vals.sort()
        y_ax_upper += len(c_silhouette_vals)
        color= cm.jet(float(i)/n_clusters)
        plt.barh(range(y_ax_lower,y_ax_upper),c_silhouette_vals,height=1,edgecolor='none',color=color)
        yticks.append((y_ax_lower+y_ax_upper)/2.)
        y_ax_lower += len(c_silhouette_vals)
    silhouette_avg=np.mean(silhouette_vals)
    Plot1_text_s = "The Silhouette Average for {} is {}".format(clusterer, round(silhouette_avg,2))
    plt.axvline(silhouette_avg,color="red",linestyle="--")
    plt.xticks(fontsize=15)
    plt.yticks(yticks, np.unique(labels) +1, fontsize=15)
    plt.ylabel('Cluster', fontsize=15)
    plt.xlabel('silhouette coefficient', fontsize=15)
    plt.text(0.2, 0, Plot1_text_s, fontsize = 20, color ="white", bbox = {"facecolor": 'green'})
    plt.show(False)
    if clusterer == 'KMeans':
        plt.savefig(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\silhcoef\silhouette_coefficient {} #clusters  = {}.jpg'.format(clusterer, n), dpi=300)
    elif clusterer == 'DBSCAN':
        plt.savefig(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\silhcoef\silhouette_coefficient {} epsilon  = {}.jpg'.format(clusterer, epsilon), dpi=300)
    elif clusterer =='GMM':
        plt.savefig(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\silhcoef\silhouette_coefficient {} #clusters  = {}.jpg'.format(clusterer, n), dpi=300)
    else:
        print('Clusterer is Not Found')

    return round(silhouette_avg,2)

## DBSCAN

### Finding Optimum Epsilon value for DBSCAN

#### First,  Nearest Neigbour will be used to have sense about the epsilon value, which is around 0.9

In [ ]:
Neighbors = NearestNeighbors(n_neighbors = 24) # 24 neighbors( Dimension  = 12, neigh = Dimension*2)
nbrs = Neighbors.fit(DataScaled)
distances, indices = nbrs.kneighbors(DataScaled)
fig = plt.figure(figsize=(6,5))
distances = np.sort(distances, axis=0)
for i in range(24):
    plt.plot(distances[:, i])
plt.title('Finding the optimum epsilon', fontsize = 15)
plt.xlabel('Samples sorted by distance', fontsize = 15)
plt.ylabel('NN distance', fontsize = 15)
plt.axhline(y=0.9, color='k', linestyle='--')

#### Now we know the closest interval for epsilon value which I will use [0.5, 2.5] to calculate sensitivity of Silhouette Coefficient respect to Epsilon Values

#### Implementing DBSCSAN on Scaled Data

In [ ]:
silhouette_avg_list_DBSCAN = []
n_unique_labels = []
for i in np.linspace(0.5, 2.5, 20):
    DBSCANclusterer_Data = {'name':'DBSCAN', 'eps': i, 'min_samples': 24, 'metric':'euclidean'}
    mm = WellLog_Based_Predictor(DBSCANclusterer_Data)
    DB_Labels = mm.Clusterer(INDataScaled)
    silhouette_avg_list_DBSCAN.append(silhouette_coeff(DB_Labels, None, i, INDataScaled,'DBSCAN'))
    n_unique_labels.append(np.unique(DB_Labels).shape[0])

In [ ]:
plt.figure(figsize=(15, 10))
eps = np.linspace(0.5, 2.5, 20)
plt.plot(eps, silhouette_avg_list_DBSCAN, 'r-o')
plt.xlabel('Epsilon', fontsize=18-2)
plt.ylabel('Silhouette Coefficient', fontsize=18)
plt.xticks(fontsize=18-2)
plt.yticks(fontsize=18-2)
for i, txt in enumerate(n_unique_labels):
    plt.text(eps[i], silhouette_avg_list_DBSCAN[i], txt, fontsize = 12)
plt.savefig('(DBSCAN) Silhouette Coefficient_vs_EPS_VS_#clusts.jpg', dpi=300)

## KMeans

### Choosing optimum number of clusters

#### Again to have sense about the number of optimum clusters for KMeans I have used Dendrogram

In [ ]:
plt.figure(figsize=(15, 10), dpi = 300)
dendogram = shc.dendrogram(shc.linkage(INDataScaled, method='ward'))
plt.xlabel('Samples')
plt.ylabel('Distance')
plt.title("Dendrogram")
plt.savefig(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3\dendogram.png')

#### Based on the Dendrogram, we cannot say exact cluster numbers ( Maybe 6-7), That is why I will calculate sensitivity of Silhoette Coefficient and Sum of Squared Errors respect to Number of Clusters

In [ ]:
ERROR=[]
KMeans_silhouette_avg_list = []
for i in np.linspace(3, 20, 18):
    KMeansclusterer_Data = {'name' : 'KMeans', 'n_clusters': int(i), 'random_state': 1000, 'init': 'k-means++', 'n_init':1000, 'max_iter':500}
    mmm = WellLog_Based_Predictor(KMeansclusterer_Data)
    centers, KM_Labels, inertia = mmm.Clusterer(DataScaled)
    KMeans_silhouette_avg_list.append(silhouette_coeff(KM_Labels, i, None, DataScaled,'KMeans'))
    ERROR.append(inertia)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(np.linspace(3, 20, 18), ERROR, 'r-o')
plt.xlabel('Number of Clusters', fontsize=18-2)
plt.ylabel('Within Cluster Sum of Squared Errors', fontsize=18)
plt.xticks(np.linspace(1,21, 21), fontsize=18-2)
plt.yticks(fontsize=18-2)
plt.savefig('KMeans_SumOfSqErr_#clusters.jpg', dpi=300)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(np.linspace(3, 20, 18), KMeans_silhouette_avg_list, 'r-o')
plt.xlabel('Number of Clusters', fontsize=18-2)
plt.ylabel('Silhouette Coefficient', fontsize=18)
plt.xticks(np.linspace(1,21, 21), fontsize=18-2)
plt.yticks(fontsize=18-2)
plt.savefig('(KMeans)Silhouette Coefficient_vs_#clusters.jpg', dpi=300)

## GMM

### To be more sure, I decided to use GMM ( All procedure is the same as KMeans)

In [ ]:
GMM_silhouette_avg_list = []
for i in np.linspace(3, 20, 18):
    GMMclusterer_Data = {'name' : 'GMM', 'n_clusters': int(i)}
    mmmm = WellLog_Based_Predictor(GMMclusterer_Data)
    GMM_Labels = mmmm.Clusterer(INDataScaled)
    GMM_silhouette_avg_list.append(silhouette_coeff(GMM_Labels, i, None, INDataScaled,'GMM'))
    ERROR.append(inertia)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(np.linspace(3, 20, 18), GMM_silhouette_avg_list, 'r-o')
plt.xlabel('Number of Clusters', fontsize=18-2)
plt.ylabel('Silhouette Coefficient', fontsize=18)
plt.xticks(np.linspace(1,21, 21), fontsize=18-2)
plt.yticks(fontsize=18-2)
plt.savefig('(GMM)Silhouette Coefficient_vs_#clusters.jpg', dpi=300)

## Summary

DBSCAN: Based on the Elbow method fron NearestNeighbour  0.9  with 7 clusters has been chosen as appropriate epsilon value. In silhouette results 0.9 epsilon corresponds to 0.2 (it is lower than 0.2). But later we can see that it increases. The maximum silhoette walue we got for DBSCAN is 0.6. However when we analyse silhouette graph we can see that data has been clustered in 2 clusters with about 1 to 9 ratio which does not haver any meaning

KMeans: From the elbow method if we assume that cluster values are in the interval from 3 to 11, we got highest silhouette coeff 0.33 for 3. But the size of clusters are relatively high thats why in my opinion the cluster value which satisfies both elbow value and Silhoutte coeff is 7.

GMM: Again same as KMeans we got highest silhouette coeff 0.3 for 3

Eventually,  KMeans with n_clusters = 7

UPDATE: Based on the previous results I decided to decrease the number of clusters to 3. The main reason is while training the regression models due to less number of data points in some clusters models cannot predict well.

### Labeling

In [ ]:
DataScaled_df = pd.DataFrame(INDataScaled, columns=Data.columns[0:-1])

In [ ]:
DataScaled_df["Water Saturation (SW)"] = OUTDataScaled

In [ ]:
KMeansclusterer_Data = {'name' : 'KMeans', 'n_clusters': int(3), 'random_state': 1000, 'init': 'k-means++', 'n_init':1000, 'max_iter':500}
mmm = WellLog_Based_Predictor(KMeansclusterer_Data)
centers, KM_Labels, inertia = mmm.Clusterer(INDataScaled) # We have to cluster our data based on first 11 input data

In [ ]:
KM_DataScaled = DataScaled_df.copy()
KM_Data_Manipulated = Data_Manipulated.copy()
KM_DataScaled['Labels'] = KM_Labels
KM_Data_Manipulated['Labels'] = KM_Labels

In [ ]:
KM_DataScaled.to_csv(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\KM_DataScaled.csv', index=False)
KM_Data_Manipulated.to_csv(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\KM_Data_Manipulated.csv', index=False)

# Question 2: Use atleast two supervised classification techniques (Random Forest, Gradient Boosting or KNN) to classify the dataset based on the appropriate number of clusters determined in Question 1. Use cluster as labels to train classification algorithm. with relevant well logs as inputs

Provide Confusion matrix, recall, precision and F1 score.

In [ ]:
KM_DataScaled = pd.read_csv(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\KM_DataScaled.csv')
KM_DataScaled_TRAIN, KM_DataScaled_TEST = train_test_split(KM_DataScaled, train_size=0.95, random_state=42)
Inputs  = KM_DataScaled_TRAIN.drop(["Water Saturation (SW)", 'Labels'],axis=1) # droping output variable from the input dataset
Outputs = KM_DataScaled_TRAIN['Labels'] # Assigning output variable
X_train, X_test, y_train, y_test = train_test_split(Inputs, Outputs, test_size=0.30, random_state=42)

KM_Data_Manipulated = pd.read_csv(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\KM_Data_Manipulated.csv')
KM_Data_Manipulated_TRAIN, KM_Data_Manipulated_TEST = train_test_split(KM_Data_Manipulated, train_size=0.95, random_state=42)
Inputs_NotScaled  = KM_Data_Manipulated_TRAIN.drop(['Labels'],axis=1) # droping output variable from the input dataset
Outputs_NotScaled = KM_Data_Manipulated_TRAIN['Labels'] # Assigning output variable
X_train_NotScaled, X_test_NotScaled, y_train_NotScaled, y_test_NotScaled = train_test_split(Inputs_NotScaled, Outputs_NotScaled, test_size=0.30, random_state=42)

In [ ]:
def ClassificationResults(y_pred, y_test, classifier):
    ConfMat = confusion_matrix(y_test, y_pred)
    print(' Results for {} classifier'.format(classifier))
    print('Accuracy Score:',accuracy_score(y_test, y_pred))
    print('Confusion Matrix:')
    print(ConfMat)
    print('Classification Report')
    print(classification_report(y_test, y_pred))
    sns.heatmap(ConfMat, center=True, annot=True, cmap='turbo', linewidths=3, linecolor='black', fmt = ".1f")
    plt.show()

## Random Forest

Initalize and Fit the Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
seed = 42
np.random.seed(seed)
RF = RandomForestClassifier(n_estimators=5000, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=2).fit(X_train, y_train)

Save and Load Model

In [ ]:
# save the model to a file
dump(RF, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\RF_clssfr.joblib')

# load the saved model from a file
RF = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\RF_clssfr.joblib')

Predict and Visualzie Results

In [ ]:
y_pred = RF.predict(X_test)
ClassificationResults(y_pred, y_test, 'Random Forest')

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
seed = 42
np.random.seed(seed)
GB =GradientBoostingClassifier(learning_rate=0.1, n_estimators=2000, criterion='friedman_mse',min_samples_split=2, min_samples_leaf=1, max_depth=3,max_features=None).fit(X_train, y_train)


Save and Load Model

In [ ]:
# save the model to a file
dump(GB, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\GB_clssfr.joblib')

# load the saved model from a file
GB = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\GB_clssfr.joblib')

Predict and Visualzie Results

In [ ]:
y_pred = GB.predict(X_test)
ClassificationResults(y_pred, y_test, 'Gradient Boosting')

## Summary

Both classifier has made good results with the Accuracy= 98%

In [ ]:
pipe_RF = make_pipeline(INscaler, RF)
pipe_GB = make_pipeline(INscaler, GB)

# Question 3: Train atleast two supervised machine learning regression techniques (RF, GB, KNN, DT, or any other) on each cluster separately to predict the water saturation as label with relevant well logs as inputs.

Provide training testing crossplots, RMSE, and R2 score for each model corresponds to each cluster.

In [ ]:
def data_cluster_masking(data, n_clusters, target):
    mask_dict = {}
    for i in range(n_clusters):
        mask_dict[i] = data[target] == i
    return mask_dict

In [ ]:
Inputs  = np.array(KM_Data_Manipulated_TRAIN.drop(['Water Saturation (SW)', 'Labels'],axis=1))     # droping output variable from the input dataset
Outputs = np.array(KM_Data_Manipulated_TRAIN['Water Saturation (SW)']).reshape((-1,1))   # Assigning output variable

# Adding Scaler module for each inputs and Outputs for inverse transformation

mlp_In_scaler = StandardScaler()
mlp_In_scaler.fit(Inputs)
dump(mlp_In_scaler, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\mlp_In_scaler.joblib')
mlp_Out_scaler = StandardScaler()
mlp_Out_scaler.fit(Outputs)
dump(mlp_Out_scaler, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\mlp_Out_scaler.joblib')

In [ ]:
mlp_In_scaler = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\mlp_In_scaler.joblib')
Inputs_scaled = mlp_In_scaler.transform(Inputs)

mlp_Out_scaler = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\mlp_Out_scaler.joblib')
Outputs_scaled = mlp_Out_scaler.transform(Outputs)

In [ ]:
Inputs_scaled_splitted = {}
def data_cluster_splitting(Inputs, Outputs, data, n_clusters, target):
    dict_splitted_Input_data = {}
    dict_splitted_Output_data = {}
    masks = data_cluster_masking(data, n_clusters, target)
    for i in range (n_clusters):
        dict_splitted_Input_data[i] = Inputs[masks[i]]
        dict_splitted_Output_data[i] = Outputs[masks[i]]
    return dict_splitted_Input_data, dict_splitted_Output_data

In [ ]:
def train_test_split_clustered_data(In, Out, n_clusters):
    dic ={}
    for i in range(n_clusters):
        dic[i] = train_test_split(In[i], Out[i], test_size=0.30, random_state=42)
    return dic

In [ ]:
IN_data, OUT_data = data_cluster_splitting(Inputs, Outputs, KM_Data_Manipulated_TRAIN, 3, target = 'Labels')


In [ ]:
train_test_splitted_data = train_test_split_clustered_data(IN_data, OUT_data, 3)

In [ ]:
def R2_pairity_RMSE(cluster_name,model_name,y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes):
    """R2 score RMSE and pairity plot calculation"""
    # Get prediciton score in terms of R2
    r2_test = r2_score(y_test_true, y_test_pred)
    r2_train = r2_score(y_train_true, y_train_pred)

    # Plot parity plots
    fs = 18
    plt.figure(figsize=(10, 25))

    plt.suptitle(f'$SW$ Parity Plot', fontsize=fs)
    plt.subplot(3, 1, 1)

    plt.plot(y_test_true, y_test_pred, 'ro', label=f'Test: R2 = {r2_test: .3f}')
    plt.plot([y_test_true.min(), y_test_true.max()], [y_test_true.min(), y_test_true.max()], 'k-.')
    plt.ylabel('Prediction, SW', fontsize=fs)

    plt.legend(fontsize=fs,edgecolor='black')

    plt.subplot(3, 1, 2)
    plt.plot(y_train_true, y_train_pred, 'b^', label=f'Train: R2 = {r2_train: .3f}')
    plt.plot([y_train_true.min(), y_train_true.max()], [y_train_true.min(), y_train_true.max()], 'k-.')

    plt.xlabel('True, SW', fontsize=fs)
    plt.ylabel('Prediction, SW', fontsize=fs)
    plt.legend(fontsize=fs, edgecolor='black')

    plt.subplot(3, 1, 3)
    train_RMSEmean = np.mean(train_RMSEscores, axis=1)
    test_RMSEmean = np.mean(test_RMSEscores, axis=1)
    plt.plot(train_sample_sizes, -train_RMSEmean, color="k",label="Training RMSE score")
    plt.plot(train_sample_sizes, -test_RMSEmean,'--', color="k",label="Cross-validation RMSE score")
    plt.title("Learning Curve for BHP Data Set with DT")
    plt.xlabel("Training Set Sample Size"),
    plt.ylabel("RMSE Score"),
    plt.legend(loc="best")

    plt.tight_layout()
    plt.savefig(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Pairity Plots\R2_parity_RMSE_plots_SW_{}_{}.png'.format(cluster_name,model_name))

## Random Forest Regressor

In [ ]:
def estimator_sensitivity_RF(cluster_name, train_test_splitted_data, estimator_list):
    """ this function is used to find optimal number of estimators for Random Forest Regressor"""
    seed=42
    n = cluster_name
    np.random.seed(seed)
    X_train_scaled = train_test_splitted_data[n][0]
    y_train_scaled = train_test_splitted_data[n][2]
    X_test_scaled  = train_test_splitted_data[n][1]
    y_test_scaled  = train_test_splitted_data[n][3]
    lstr2 = []
    for i in estimator_list:
        model    = RandomForestRegressor(n_estimators=int(i), max_depth=115,
                                        min_samples_split=5,
                                        random_state=42)
        model.fit(X_train_scaled,y_train_scaled.flatten())

        y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
        y_test_pred = mlp_Out_scaler.inverse_transform(model.predict(X_test_scaled).reshape(-1, 1))
        lstr2.append(r2_score(y_test_true, y_test_pred))
    return lstr2

### Cluster 0

In [ ]:
n = 1
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

In [ ]:
cluster_name = 0
estimator_list = np.linspace(65, 100, 36)
lst = estimator_sensitivity_RF(cluster_name, train_test_splitted_data, estimator_list)
plt.figure(figsize=(10, 5))
plt.plot(np.linspace(65, 100, 36), lst, 'r-o')
plt.xticks(np.linspace(65, 100, 36))
plt.grid()
plt.xlabel('# ESTIMATOR')
plt.ylabel('R2 SCORE FOR TEST')

In [ ]:
seed=42
np.random.seed(seed)
n = 0
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

model_RFregr_0    = RandomForestRegressor(n_estimators=98, max_depth=115,
                                  min_samples_split=5,
                                  random_state=42)
dump(model_RFregr_0, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_RFregr_0.joblib')
model_RFregr_0 = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_RFregr_0.joblib')
model_RFregr_0.fit(X_train_scaled,y_train_scaled.flatten())

y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
y_train_true = mlp_Out_scaler.inverse_transform(y_train_scaled)
y_test_pred = mlp_Out_scaler.inverse_transform(model_RFregr_0.predict(X_test_scaled).reshape(-1, 1))
y_train_pred = mlp_Out_scaler.inverse_transform(model_RFregr_0.predict(X_train_scaled).reshape(-1, 1))

train_sample_sizes, train_RMSEscores,test_RMSEscores = learning_curve(model_RFregr_0,IN_data[n],OUT_data[n].flatten(),cv=10,
                                                                    scoring='neg_root_mean_squared_error',
                                                                    train_sizes=np.linspace(0.1,1.0,5),
                                                                    random_state=42)

R2_pairity_RMSE(n,"RF_regr",y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes)

### Cluster 1

In [ ]:
cluster_name = 1
estimator_list = np.linspace(65, 100, 36)
lst = estimator_sensitivity_RF(cluster_name, train_test_splitted_data, estimator_list)
plt.figure(figsize=(10, 5))
plt.plot(np.linspace(65, 100, 36), lst, 'r-o')
plt.xticks(np.linspace(65, 100, 36))
plt.grid()
plt.xlabel('# ESTIMATOR')
plt.ylabel('R2 SCORE FOR TEST')

In [ ]:
seed=42
np.random.seed(seed)
n = 1
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

model_RFregr_1    = RandomForestRegressor(n_estimators=65, max_depth=115,
                                  min_samples_split=5,
                                  random_state=42)
dump(model_RFregr_1, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_RFregr_1.joblib')
model_RFregr_1 = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_RFregr_1.joblib')
model_RFregr_1.fit(X_train_scaled,y_train_scaled.flatten())

y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
y_train_true = mlp_Out_scaler.inverse_transform(y_train_scaled)
y_test_pred = mlp_Out_scaler.inverse_transform(model_RFregr_1.predict(X_test_scaled).reshape(-1, 1))
y_train_pred = mlp_Out_scaler.inverse_transform(model_RFregr_1.predict(X_train_scaled).reshape(-1, 1))

train_sample_sizes, train_RMSEscores,test_RMSEscores = learning_curve(model_RFregr_1,IN_data[n],OUT_data[n].flatten(),cv=10,
                                                                    scoring='neg_root_mean_squared_error',
                                                                    train_sizes=np.linspace(0.1,1.0,5),
                                                                    random_state=42)

R2_pairity_RMSE(n,"RF_regr",y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes)

### Cluster 2

In [ ]:
cluster_name = 2
estimator_list = np.linspace(65, 130, 66)
lst = estimator_sensitivity_RF(cluster_name, train_test_splitted_data, estimator_list)
plt.figure(figsize=(10, 5))
plt.plot(estimator_list, lst, 'r-o')
plt.xticks(estimator_list)
plt.grid()
plt.xlabel('# ESTIMATOR')
plt.ylabel('R2 SCORE FOR TEST')

![image.png](attachment:image.png)

In [ ]:
seed=42
np.random.seed(seed)
n = 2
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

model_RFregr_2    = RandomForestRegressor(n_estimators=108, max_depth=115,
                                  min_samples_split=5,
                                  random_state=42)
dump(model_RFregr_2, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_RFregr_2.joblib')
model_RFregr_2 = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_RFregr_2.joblib')
model_RFregr_2.fit(X_train_scaled,y_train_scaled.flatten())

y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
y_train_true = mlp_Out_scaler.inverse_transform(y_train_scaled)
y_test_pred = mlp_Out_scaler.inverse_transform(model_RFregr_2.predict(X_test_scaled).reshape(-1, 1))
y_train_pred = mlp_Out_scaler.inverse_transform(model_RFregr_2.predict(X_train_scaled).reshape(-1, 1))

train_sample_sizes, train_RMSEscores,test_RMSEscores = learning_curve(model_RFregr_2,IN_data[n],OUT_data[n].flatten(),cv=10,
                                                                    scoring='neg_root_mean_squared_error',
                                                                    train_sizes=np.linspace(0.1,1.0,5),
                                                                    random_state=42)

R2_pairity_RMSE(n,"RF_regr",y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes)

### It is observed from the R2 scores of 7 models, most of the model have relatively satisfactory R2 scores. Less R2 scores is the result of less number of train data for the cluster, in other words model cannot be trained well

## Gradient Boosting Regressor

In [ ]:
def estimator_sensitivity_GB(cluster_name, train_test_splitted_data, estimator_list):
    """ this function is used to find optimal number of estimators for Gradient Boosting Regressor"""
    seed=42
    n = cluster_name
    np.random.seed(seed)
    X_train_scaled = train_test_splitted_data[n][0]
    y_train_scaled = train_test_splitted_data[n][2]
    X_test_scaled  = train_test_splitted_data[n][1]
    y_test_scaled  = train_test_splitted_data[n][3]
    lstr2 = []
    for i in estimator_list:
        model = GradientBoostingRegressor(n_estimators=int(i), learning_rate=0.01, max_depth=5,
                                            random_state=48,
                                            loss='squared_error')
        model.fit(X_train_scaled,y_train_scaled.flatten())

        y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
        y_test_pred = mlp_Out_scaler.inverse_transform(model.predict(X_test_scaled).reshape(-1, 1))
        lstr2.append(r2_score(y_test_true, y_test_pred))
    return lstr2

### Cluster 0

In [ ]:
cluster_name = 0
estimator_list = np.linspace(600, 900, 26)
lst = estimator_sensitivity_GB(cluster_name, train_test_splitted_data, estimator_list)
plt.figure(figsize=(10, 5))
plt.plot(estimator_list, lst, 'r-o')
plt.xticks(estimator_list)
plt.grid()
plt.xlabel('# ESTIMATOR')
plt.ylabel('R2 SCORE FOR TEST')

In [ ]:
seed=42
np.random.seed(seed)
n = 0
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

model_GBregr_0 = GradientBoostingRegressor(n_estimators=900, learning_rate=0.01, max_depth=5,
                                    random_state=48,
                                    loss='squared_error')
dump(model_GBregr_0, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_GBregr_0.joblib')
model_GBregr_0 = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_GBregr_0.joblib')
model_GBregr_0.fit(X_train_scaled,y_train_scaled.flatten())

y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
y_train_true = mlp_Out_scaler.inverse_transform(y_train_scaled)
y_test_pred = mlp_Out_scaler.inverse_transform(model_GBregr_0.predict(X_test_scaled).reshape(-1, 1))
y_train_pred = mlp_Out_scaler.inverse_transform(model_GBregr_0.predict(X_train_scaled).reshape(-1, 1))

train_sample_sizes, train_RMSEscores,test_RMSEscores = learning_curve(model_GBregr_0,IN_data[n],OUT_data[n].flatten(),cv=10,
                                                                    scoring='neg_root_mean_squared_error',
                                                                    train_sizes=np.linspace(0.1,1.0,5),
                                                                    random_state=42)

R2_pairity_RMSE(n,"GB_regr",y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes)

### Cluster 1

In [ ]:
cluster_name = 1
estimator_list = np.linspace(600, 900, 26)
lst = estimator_sensitivity_RF(cluster_name, train_test_splitted_data, estimator_list)
plt.figure(figsize=(10, 5))
plt.plot(estimator_list, lst, 'r-o')
plt.xticks(estimator_list)
plt.grid()
plt.xlabel('# ESTIMATOR')
plt.ylabel('R2 SCORE FOR TEST')

In [ ]:
seed=42
np.random.seed(seed)
n = 1
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

model_GBregr_1    = GradientBoostingRegressor(n_estimators=600, learning_rate=0.01, max_depth=5,
                                    random_state=48,
                                    loss='squared_error')
dump(model_GBregr_1, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_GBregr_1.joblib')
model_GBregr_1 = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_GBregr_1.joblib')
model_GBregr_1.fit(X_train_scaled,y_train_scaled.flatten())

y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
y_train_true = mlp_Out_scaler.inverse_transform(y_train_scaled)
y_test_pred = mlp_Out_scaler.inverse_transform(model_GBregr_1.predict(X_test_scaled).reshape(-1, 1))
y_train_pred = mlp_Out_scaler.inverse_transform(model_GBregr_1.predict(X_train_scaled).reshape(-1, 1))

train_sample_sizes, train_RMSEscores,test_RMSEscores = learning_curve(model_GBregr_1,IN_data[n],OUT_data[n].flatten(),cv=10,
                                                                    scoring='neg_root_mean_squared_error',
                                                                    train_sizes=np.linspace(0.1,1.0,5),
                                                                    random_state=42)

R2_pairity_RMSE(n,"GB_regr",y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes)

### Cluster 2

In [ ]:
cluster_name = 2
estimator_list = np.linspace(600, 900, 26)
lst = estimator_sensitivity_RF(cluster_name, train_test_splitted_data, estimator_list)
plt.figure(figsize=(10, 5))
plt.plot(estimator_list, lst, 'r-o')
plt.xticks(estimator_list)
plt.grid()
plt.xlabel('# ESTIMATOR')
plt.ylabel('R2 SCORE FOR TEST')

In [ ]:
seed=42
np.random.seed(seed)
n = 2
X_train_scaled = train_test_splitted_data[n][0]
y_train_scaled = train_test_splitted_data[n][2]
X_test_scaled  = train_test_splitted_data[n][1]
y_test_scaled  = train_test_splitted_data[n][3]

model_GBregr_2    = GradientBoostingRegressor(n_estimators=672, learning_rate=0.01, max_depth=5,
                                    random_state=48,
                                    loss='squared_error')
dump(model_GBregr_2, r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_GBregr_2.joblib')
model_GBregr_2 = load(r'C:\Users\YUSIFOH\ERPE 394A\Assignment 3 copy\Saved Models\model_GBregr_2.joblib')
model_GBregr_2.fit(X_train_scaled,y_train_scaled.flatten())

y_test_true = mlp_Out_scaler.inverse_transform(y_test_scaled)
y_train_true = mlp_Out_scaler.inverse_transform(y_train_scaled)
y_test_pred = mlp_Out_scaler.inverse_transform(model_GBregr_2.predict(X_test_scaled).reshape(-1, 1))
y_train_pred = mlp_Out_scaler.inverse_transform(model_GBregr_2.predict(X_train_scaled).reshape(-1, 1))

train_sample_sizes, train_RMSEscores,test_RMSEscores = learning_curve(model_GBregr_2,IN_data[n],OUT_data[n].flatten(),cv=10,
                                                                    scoring='neg_root_mean_squared_error',
                                                                    train_sizes=np.linspace(0.1,1.0,5),
                                                                    random_state=42)

R2_pairity_RMSE(n,"GB_regr",y_train_true, y_train_pred, y_test_true, y_test_pred, train_RMSEscores, test_RMSEscores, train_sample_sizes)

# Question 4: Develop a python code that takes well logs as input and determine the relevant cluster and predict the water saturation as an output.

In [ ]:
def predictor( classifier, regressor, data):
    if classifier == "GB":
        pipe = make_pipeline(INscaler, GB)
    elif classifier == "RF":
        pipe = make_pipeline(INscaler, RF)

    classes = pipe.predict(data)
    data = np.array(data)
    if regressor == "RF":
        results = []
        for i in range(len(classes)):
            if classes[i] == 0:
                pipe_r = make_pipeline(mlp_In_scaler, model_RFregr_0)
                Sw = mlp_Out_scaler.inverse_transform((pipe_r.predict(data[i].reshape((1,-1)))).reshape((1,-1)))
                results.append(Sw)
            elif classes[i] == 1:
                pipe_r = make_pipeline(mlp_In_scaler, model_RFregr_1)
                Sw = mlp_Out_scaler.inverse_transform((pipe_r.predict(data[i].reshape((1,-1)))).reshape((1,-1)))
                results.append(Sw)
            elif classes[i] == 2:
                pipe_r = make_pipeline(mlp_In_scaler, model_RFregr_2)
                Sw = mlp_Out_scaler.inverse_transform((pipe_r.predict(data[i].reshape((1,-1)))).reshape((1,-1)))
                results.append(Sw)

    elif regressor == "GB":
        results = []
        for i in range(len(classes)):
            if classes[i] == 0:
                pipe_r = make_pipeline(mlp_In_scaler, model_GBregr_0)
                Sw = mlp_Out_scaler.inverse_transform((pipe_r.predict(data[i].reshape((1,-1)))).reshape((1,-1)))
                results.append(Sw)
            elif classes[i] == 1:
                pipe_r = make_pipeline(mlp_In_scaler, model_GBregr_1)
                Sw = mlp_Out_scaler.inverse_transform((pipe_r.predict(data[i].reshape((1,-1)))).reshape((1,-1)))
                results.append(Sw)
            elif classes[i] == 2:
                pipe_r = make_pipeline(mlp_In_scaler, model_GBregr_2)
                Sw = mlp_Out_scaler.inverse_transform((pipe_r.predict(data[i].reshape((1,-1)))).reshape((1,-1)))
                results.append(Sw)
    return classes, results

In [ ]:
user_input = KM_Data_Manipulated_TEST.drop(['Water Saturation (SW)', 'Labels'], axis = 1)  # user input
classes_true = np.array(KM_Data_Manipulated_TEST['Labels'])
Sw_true =np.array(KM_Data_Manipulated_TEST['Water Saturation (SW)'])

In [ ]:
class_pred = predictor( 'GB', "GB", user_input)[0]
Sw_pred = predictor( 'GB', "GB", user_input)[1]

In [ ]:
true = 0
for i in range(len(classes_true)):
    if classes_true[i] == class_pred[i]:
        true += 1
acc_class = (true/len(classes_true)) * 100
acc_class

In [ ]:
acc_sw = np.average((Sw_true - Sw_pred)**2)
acc_sw